# Segmenting and Clustering Neighborhoods in Toronto

##### By Peter J. Lindner Ph.D.

### PART 1 of 3

1.  Start by creating a new Notebook for this assignment.

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'html5lib')

table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

df.head(10)

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
8,M8A,Not assigned,Not assigned
9,M9A,Downtown Toronto,Queen's Park


3. To create the above dataframe:

    * The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

    * Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

    * More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
    * If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
    * Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
    * In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [3]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

df2 = df[~df.Borough.isin(['Not assigned'])]
df2.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [4]:
# More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will 
# notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined 
# into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

df3 = df2.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df3.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [5]:
df3.loc[df3['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [6]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
# So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will
# be Queen's Park.

df3.loc[df3['Neighborhood']=="Not assigned", 'Neighborhood'] = df3.loc[df3['Neighborhood']=="Not assigned", 'Borough']

In [7]:
df_final = df3
df_final.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df_final.loc[df_final['Borough'] == "Queen's Park"]

,Postcode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [9]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

df_final.shape

(103, 3)

### PART 2 of 3

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [10]:
#Get coordniates from csv file
coord_df = pd.read_csv("Geospatial_Coordinates.csv")

#rename columns
coord_df.columns = ["Postcode", "Latitude", "Longitude"]
if(coord_df.index.name != 'Postcode'):
    coord_df = coord_df.set_index('Postcode')
    
coord_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [11]:
if(df_final.index.name != 'Postcode'):
    df_final = df_final.set_index('Postcode')
df_final.head()

,Borough,Neighborhood
Postcode,,
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [12]:
df_final = df_final.join(coord_df)
df_final.head(12)

,Borough,Neighborhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
